In [9]:
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 37.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install transformers datasets evaluate accelerate

Note: you may need to restart the kernel to use updated packages.


In [19]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# Load SST-2 dataset (general sentiment)
dataset = load_dataset("glue", "sst2")

# Check structure
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [23]:
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [10]:
import transformers
print(transformers.__version__)
print(transformers.__file__)

4.53.2
/opt/anaconda3/lib/python3.12/site-packages/transformers/__init__.py


In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2
)

In [26]:
pip install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ankurzing/sentiment-analysis-for-financial-news")

print("Path to dataset files:", path)

100%|████████████████████████████████████████| 903k/903k [00:00<00:00, 7.41MB/s]

Extracting files...
Path to dataset files: /Users/rickliu/.cache/kagglehub/datasets/ankurzing/sentiment-analysis-for-financial-news/versions/5


In [36]:
import pandas as pd
from datasets import Dataset

csv_path = "/Users/rickliu/.cache/kagglehub/datasets/ankurzing/sentiment-analysis-for-financial-news/versions/5/all-data.csv"

# Read with correct delimiter and no header
df = pd.read_csv(csv_path, encoding="ISO-8859-1", names=["label", "text"], engine="python")

print(df.head())

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Train-test split
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(train_dataset[0])


      label                                               text
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...
{'label': 'neutral', 'text': 'The contract , which was signed yesterday , will run for five years and includes a two-year extension option .'}


In [52]:
label2id = {"negative": 0, "neutral": 1, "positive": 2}

def encode_labels(example):
    if isinstance(example["label"], str):  # Only map if it's a string
        example["label"] = label2id[example["label"]]
    return example

train_dataset = train_dataset.map(encode_labels)
eval_dataset = eval_dataset.map(encode_labels)
print(train_dataset[0]) ß
print(set(train_dataset['label'][:20]))  # Check first 20 labels


Map:   0%|          | 0/3876 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

{'label': 1, 'text': 'The contract , which was signed yesterday , will run for five years and includes a two-year extension option .'}
{0, 1, 2}


In [56]:
from transformers import AutoTokenizer

model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

/opt/anaconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/3876 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

In [58]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer
)


/var/folders/ng/prfj2ls9255881q552hlmp680000gn/T/ipykernel_20544/1159644960.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [66]:
trainer.train()


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.439800


TrainOutput(global_step=729, training_loss=0.3570099609542449, metrics={'train_runtime': 573.9903, 'train_samples_per_second': 20.258, 'train_steps_per_second': 1.27, 'total_flos': 764884422282240.0, 'train_loss': 0.3570099609542449, 'epoch': 3.0})

In [68]:
model.save_pretrained("./deberta-financial")
tokenizer.save_pretrained("./deberta-financial")

('./deberta-financial/tokenizer_config.json',
 './deberta-financial/special_tokens_map.json',
 './deberta-financial/spm.model',
 './deberta-financial/added_tokens.json',
 './deberta-financial/tokenizer.json')

In [70]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="./deberta-financial", tokenizer="./deberta-financial")

post1 = "Fuck NVDA. Lost 30% in one day!"
post2 = "My dad invested 200k in Nvidia and plans to hold for 3 years."

print(sentiment_pipeline([post1, post2]))


Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.9875481128692627}, {'label': 'LABEL_1', 'score': 0.9957257509231567}]


In [72]:
import re

def extract_tickers(text):
    return re.findall(r'\b[A-Z]{2,5}\b', text)


In [74]:
extract_tickers("Fuck NVDA. Lost 30%! Maybe TSLA is next?")

['NVDA', 'TSLA']

In [84]:
 import kagglehub

# Download sarcasm dataset
sarcasm_path = kagglehub.dataset_download("rmisra/news-headlines-dataset-for-sarcasm-detection")
print("Path to sarcasm dataset:", sarcasm_path)

100%|██████████████████████████████████████| 3.30M/3.30M [00:00<00:00, 16.1MB/s]

Extracting files...
Path to sarcasm dataset: /Users/rickliu/.cache/kagglehub/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection/versions/2


In [86]:
import pandas as pd
from datasets import Dataset

csv_path = sarcasm_path + "/Sarcasm_Headlines_Dataset.json"

# Load JSON dataset
df = pd.read_json(csv_path, lines=True)

# Keep only headline & is_sarcastic
df = df[["headline", "is_sarcastic"]].rename(columns={"headline": "text", "is_sarcastic": "label"})

# Convert to HuggingFace Dataset
sarcasm_dataset = Dataset.from_pandas(df)

# Train-test split
sarcasm_dataset = sarcasm_dataset.train_test_split(test_size=0.2)
train_dataset = sarcasm_dataset["train"]
eval_dataset = sarcasm_dataset["test"]

print(train_dataset[0])


{'text': 'zayn malik breaks his twitter silence to thank fans', 'label': 0}


In [88]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/21367 [00:00<?, ? examples/s]

Map:   0%|          | 0/5342 [00:00<?, ? examples/s]

In [90]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="./sarcasm_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer
)

trainer.train()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/ng/prfj2ls9255881q552hlmp680000gn/T/ipykernel_20544/164424720.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.344500
1000,0.254300
1500,0.195600
2000,0.136800
2500,0.130300
3000,0.087500
3500,0.070600
4000,0.063800


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=4008, training_loss=0.16024079364157484, metrics={'train_runtime': 1018.5377, 'train_samples_per_second': 62.934, 'train_steps_per_second': 3.935, 'total_flos': 2122823180312064.0, 'train_loss': 0.16024079364157484, 'epoch': 3.0})

In [92]:
trainer.save_model("./sarcasm_detector")
tokenizer.save_pretrained("./sarcasm_detector")

('./sarcasm_detector/tokenizer_config.json',
 './sarcasm_detector/special_tokens_map.json',
 './sarcasm_detector/vocab.txt',
 './sarcasm_detector/added_tokens.json',
 './sarcasm_detector/tokenizer.json')

In [108]:
# Save your trained DeBERTa sentiment model
sentiment_save_path = "./deberta_sentiment"

model.save_pretrained(sentiment_save_path)
tokenizer.save_pretrained(sentiment_save_path)


('./deberta_sentiment/tokenizer_config.json',
 './deberta_sentiment/special_tokens_map.json',
 './deberta_sentiment/vocab.txt',
 './deberta_sentiment/added_tokens.json',
 './deberta_sentiment/tokenizer.json')

In [134]:
# Import necessary libraries
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import os

# Paths to local models
sentiment_model_path = os.path.abspath("./deberta_sentiment")
sarcasm_model_path = os.path.abspath("./sarcasm_detector")

print("Sentiment model path:", sentiment_model_path)
print("Sarcasm model path:", sarcasm_model_path)

# Load models and tokenizers
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_path, local_files_only=True)
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_path, local_files_only=True)

sarcasm_model = AutoModelForSequenceClassification.from_pretrained(sarcasm_model_path, local_files_only=True)
sarcasm_tokenizer = AutoTokenizer.from_pretrained(sarcasm_model_path, local_files_only=True)

# Create HuggingFace pipelines
sentiment_analyzer = pipeline("text-classification", model=sentiment_model, tokenizer=sentiment_tokenizer)
sarcasm_detector = pipeline("text-classification", model=sarcasm_model, tokenizer=sarcasm_tokenizer)

# Analyze function: Get raw model predictions
def analyze_post(text):
    sentiment = sentiment_analyzer(text)[0]
    sarcasm = sarcasm_detector(text)[0]
    
    return {
        "text": text,
        "sentiment": sentiment["label"],        # e.g., LABEL_0 / LABEL_1 / LABEL_2
        "sentiment_score": round(sentiment["score"], 4),
        "sarcasm": sarcasm["label"],            # e.g., LABEL_0 / LABEL_1
        "sarcasm_score": round(sarcasm["score"], 4)
    }

# Interpret sentiment with sarcasm logic
def interpret_sentiment_with_sarcasm(result):
    # Mapping model labels to human-readable labels
    sentiment_labels = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}
    sarcasm_labels = {"LABEL_0": "not sarcastic", "LABEL_1": "sarcastic"}
    
    # Extract values
    raw_sentiment = sentiment_labels[result["sentiment"]]
    raw_sarcasm = sarcasm_labels[result["sarcasm"]]
    sentiment_score = result["sentiment_score"]
    sarcasm_score = result["sarcasm_score"]
    
    # Apply sarcasm-based adjustment
    if sarcasm_score > 0.8 and raw_sentiment == "negative":
        final_interpretation = "positive (sarcastic bullish)"
    else:
        final_interpretation = raw_sentiment
    
    # Return structured result
    return {
        "text": result["text"],
        "raw_sentiment": raw_sentiment,
        "raw_sarcasm": raw_sarcasm,
        "sentiment_score": sentiment_score,
        "sarcasm_score": sarcasm_score,
        "final_interpretation": final_interpretation
    }

Device set to use mps:0


Sentiment model path: /Users/rickliu/Desktop/deberta_sentiment
Sarcasm model path: /Users/rickliu/Desktop/sarcasm_detector


Device set to use mps:0


In [138]:
from IPython.display import Image

# Display the image
Image(filename='screenshot.png')


post1 = '''OPENDOOR - this isn't just a pump. 
People have been in since IPOB. Well, have you been in IPOB so early that you were trading IPOB for profit on the low volume? 
Well, I was, since the early, early days. THEN it was announced that IPOB = Opendoor, 
which I took as a divine sign, as I'm deeply involved in Real Estate and saw the problem they solved. 
Five fucking years I've been in this thing and looked at it inside out, while averaging down.

Fast forward to today. The problem has not been solved but they are making strides to get there. 
They realized that can't just go in there guns blazing. They need some Vaseline and a John Mayer, 
Sade, Keith Sweat and Juvenile Slow Motion playlist.

Buckle up. I'd eat my dick if OPENDOOR doesn't compound this run with positive news. 
Just buy in. Or don't. IDC, as I am tipsy right now. Deservedly so.
'''


# Run analysis
raw_result = analyze_post(post1)
output = interpret_sentiment_with_sarcasm(raw_result)

print("\n🔍 Final Output:")
for key, value in output.items():
    print(f"{key}: {value}")



🔍 Final Output:
text: OPENDOOR - this isn't just a pump. 
People have been in since IPOB. Well, have you been in IPOB so early that you were trading IPOB for profit on the low volume? 
Well, I was, since the early, early days. THEN it was announced that IPOB = Opendoor, 
which I took as a divine sign, as I'm deeply involved in Real Estate and saw the problem they solved. 
Five fucking years I've been in this thing and looked at it inside out, while averaging down.

Fast forward to today. The problem has not been solved but they are making strides to get there. 
They realized that can't just go in there guns blazing. They need some Vaseline and a John Mayer, 
Sade, Keith Sweat and Juvenile Slow Motion playlist.

Buckle up. I'd eat my dick if OPENDOOR doesn't compound this run with positive news. 
Just buy in. Or don't. IDC, as I am tipsy right now. Deservedly so.
raw_sentiment: negative
raw_sarcasm: not sarcastic
sentiment_score: 0.9981
sarcasm_score: 0.9981
final_interpretation: posit

In [140]:
# Test
post2 = '''
Who else is investing in the MAMACITA portfolio?

The MAMACITA portfolio is working really well for me. It consists of:

Microsoft

Alphabet

Meta

Amazon

Costco

Invidia (Nvidia is named after the Latin 'invidia', which means 'envy'. This is what makes this portfolio a latina 💃)

Tesla

Apple

I feel like these holdings say it all. If you're not investing in a latina portfolio you're missing out
'''


# Run analysis
raw_result = analyze_post(post2)
output = interpret_sentiment_with_sarcasm(raw_result)

print("\n🔍 Final Output:")
for key, value in output.items():
    print(f"{key}: {value}")



🔍 Final Output:
text: 
Who else is investing in the MAMACITA portfolio?

The MAMACITA portfolio is working really well for me. It consists of:

Microsoft

Alphabet

Meta

Amazon

Costco

Invidia (Nvidia is named after the Latin 'invidia', which means 'envy'. This is what makes this portfolio a latina 💃)

Tesla

Apple

I feel like these holdings say it all. If you're not investing in a latina portfolio you're missing out

raw_sentiment: negative
raw_sarcasm: not sarcastic
sentiment_score: 0.9993
sarcasm_score: 0.9993
final_interpretation: positive (sarcastic bullish)
